# Complete ESIBD Pressure Chain Test

This notebook provides comprehensive control and monitoring for the complete ESIBD instrument pressure chain.

## Equipment Configuration:
- **HiScroll Pumps**: 2 units (HiScroll1, HiScroll2) as backing pumps
- **HiPace300 Pumps**: 2 units (Transfer Chamber, Deposition Chamber)
- **HiPace450 Pump**: 1 unit (main high vacuum pump)
- **TPG366**: Pressure controller with 5 pressure sensors
- **Chiller**: 1 cooling unit

## Features:
- Individual pump control (turn on/off each pump separately)
- Live monitoring with multiple synchronized plots
- Pressure monitoring (5 sensors with shared time axis)
- Temperature monitoring (HiScrolls and HiPaces separately)
- Data logging with timestamps
- Shared logging system using loguru

## 1. Import Required Libraries and Setup

In [1]:
import sys
import threading
import time
from datetime import datetime
import asyncio
from typing import Dict, Optional, Any
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
import pickle
from collections import deque

from loguru import logger
import os
from pathlib import Path

# Add path to src modules
sys.path.append(os.path.join(os.getcwd(), '..', '..', 'src'))

# Import device modules
from devices.pfeiffer.hiscroll12.hiscroll12 import HiScroll12
from devices.pfeiffer.hipacebus import HiPace300Bus
from devices.pfeiffer.tpg366.tpg366 import TPG366
from devices.chiller.chiller import Chiller

print("✅ All libraries and device modules imported successfully")

✅ All libraries and device modules imported successfully


## 2. Setup Shared Logging System

In [2]:
# Get the repository root and create shared logs directory
repo_root = Path(os.getcwd()).parent.parent
log_dir = repo_root / "debugging" / "logs"
log_dir.mkdir(parents=True, exist_ok=True)

# Create shared log file for all devices
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
shared_log_file = log_dir / f"esibd_pressure_chain_{timestamp}.log"

# Configure shared logger
logger.remove()  # Remove default logger

# Add console logger with INFO level
logger.add(sys.stderr, level="INFO", format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}")

# Add shared file logger with DEBUG level
logger.add(
    str(shared_log_file),
    level="DEBUG",
    format="{time:YYYY-MM-DD HH:mm:ss.SSS} | {level} | {name}:{function}:{line} | {message}",
    rotation="1 day",
    retention="30 days",
    compression="zip"
)

logger.info("ESIBD Pressure Chain Test system initialized with shared logging")
print(f"Repository root: {repo_root}")
print(f"Shared logs will be saved to: {shared_log_file}")

2025-10-08 15:36:00 | INFO | ESIBD Pressure Chain Test system initialized with shared logging


Repository root: C:\Users\ESIBDlab\PycharmProjects\esibd_bs
Shared logs will be saved to: C:\Users\ESIBDlab\PycharmProjects\esibd_bs\debugging\logs\esibd_pressure_chain_20251008_153600.log


## 3. Device Configuration (COM Ports & Addresses)

In [3]:
# Device configuration dictionaries
# Modify these COM ports and addresses according to your actual hardware setup

# HiScroll backing pumps
HiScroll1_dict = {
    'com_port': 'COM19',
    'device_address': 2,
    'device_type': 'HiScroll12',
    'description': 'Backing pump 1'
}

HiScroll2_dict = {
    'com_port': 'COM21',
    'device_address': 2,
    'device_type': 'HiScroll12',
    'description': 'Backing pump 2'
}

# HiPace300 turbo pumps
HiPace300_Transfer_dict = {
    'com_port': 'COM43',
    'device_address': 101,
    'omnicontrol_address': 101,
    'tc400_address': 1,
    'gauge1_address': 122,
    'device_type': 'HiPace300',
    'description': 'Transfer chamber turbo pump'
}

HiPace300_Depo_dict = {
    'com_port': 'COM37',
    'device_address': 101,
    'omnicontrol_address': 101,
    'tc400_address': 1,
    'gauge1_address': 122,
    'device_type': 'HiPace300', 
    'description': 'Deposition chamber turbo pump'
}

# HiPace450 main turbo pump (placeholder - needs device class implementation)
HiPace450_dict = {
    'com_port': 'COM34',
    'device_address': 101,
    'omnicontrol_address': 101,
    'tc400_address': 1,
    'gauge1_address': 122,
    'device_type': 'HiPace450',
    'description': 'Main high vacuum pump (initialized as HiPace300Bus)'
}

# TPG366 pressure controller with 5 sensors
TPG366_dict = {
    'com_port': 'COM22',
    'device_address': 10,
    'device_type': 'TPG366',
    'description': 'Pressure controller with 5 sensors',
    'sensor_channels': [1, 2, 3, 4, 5]
}

# Chiller cooling unit
Chiller_dict = {
    'com_port': 'COM39',
    'device_address': 1,
    'device_type': 'Chiller',
    'description': 'Cooling unit'
}

# Print configuration summary
print("=" * 60)
print("ESIBD PRESSURE CHAIN - DEVICE CONFIGURATION")
print("=" * 60)
print(f"\nHiScroll1: {HiScroll1_dict['com_port']} @ Address {HiScroll1_dict['device_address']}")
print(f"HiScroll2: {HiScroll2_dict['com_port']} @ Address {HiScroll2_dict['device_address']}")
print(f"HiPace300 Transfer: {HiPace300_Transfer_dict['com_port']} @ Address {HiPace300_Transfer_dict['device_address']}")
print(f"HiPace300 Depo: {HiPace300_Depo_dict['com_port']} @ Address {HiPace300_Depo_dict['device_address']}")
print(f"HiPace450: {HiPace450_dict['com_port']} @ Address {HiPace450_dict['device_address']} (placeholder)")
print(f"TPG366: {TPG366_dict['com_port']} @ Address {TPG366_dict['device_address']} (5 sensors)")
print(f"Chiller: {Chiller_dict['com_port']} @ Address {Chiller_dict['device_address']}")
print("\n✅ Device dictionaries configured successfully")

ESIBD PRESSURE CHAIN - DEVICE CONFIGURATION

HiScroll1: COM19 @ Address 2
HiScroll2: COM21 @ Address 2
HiPace300 Transfer: COM43 @ Address 101
HiPace300 Depo: COM37 @ Address 101
HiPace450: COM34 @ Address 101 (placeholder)
TPG366: COM22 @ Address 10 (5 sensors)
Chiller: COM39 @ Address 1

✅ Device dictionaries configured successfully


## 4. Connect Devices

In [4]:
# Initialize and connect HiScroll1
hiscroll1 = HiScroll12(
    device_id="hiscroll1",
    port=HiScroll1_dict['com_port'],
    device_address=HiScroll1_dict['device_address'],
    logger=logger
)
hiscroll1.connect()
print(f"✅ HiScroll1 connected on {HiScroll1_dict['com_port']}")

2025-10-08 15:36:09 | INFO | Connecting to Pfeiffer device hiscroll1 on COM19
2025-10-08 15:36:09 | INFO | Successfully connected to device at address 2


✅ HiScroll1 connected on COM19


In [5]:
# Initialize and connect HiScroll2
hiscroll2 = HiScroll12(
    device_id="hiscroll2",
    port=HiScroll2_dict['com_port'],
    device_address=HiScroll2_dict['device_address'],
    logger=logger
)
hiscroll2.connect()
print(f"✅ HiScroll2 connected on {HiScroll2_dict['com_port']}")

2025-10-08 15:36:11 | INFO | Connecting to Pfeiffer device hiscroll2 on COM21
2025-10-08 15:36:11 | INFO | Successfully connected to device at address 2


✅ HiScroll2 connected on COM21


In [6]:
# Initialize and connect HiPace300 Transfer
hipace_transfer = HiPace300Bus(
    device_id="hipace_transfer",
    port=HiPace300_Transfer_dict['com_port'],
    device_address=HiPace300_Transfer_dict['device_address'],
    omnicontrol_address=HiPace300_Transfer_dict['omnicontrol_address'],
    tc400_address=HiPace300_Transfer_dict['tc400_address'],
    gauge1_address=HiPace300_Transfer_dict['gauge1_address'],
    logger=logger
)
hipace_transfer.connect()
print(f"✅ HiPace300 Transfer connected on {HiPace300_Transfer_dict['com_port']}")

2025-10-08 15:36:12 | INFO | Connecting to Pfeiffer device hipace_transfer on COM43
2025-10-08 15:36:12 | INFO | Successfully connected to device at address 101


✅ HiPace300 Transfer connected on COM43


In [7]:
# Initialize and connect HiPace300 Depo
hipace_depo = HiPace300Bus(
    device_id="hipace_depo",
    port=HiPace300_Depo_dict['com_port'],
    device_address=HiPace300_Depo_dict['device_address'],
    omnicontrol_address=HiPace300_Depo_dict['omnicontrol_address'],
    tc400_address=HiPace300_Depo_dict['tc400_address'],
    gauge1_address=HiPace300_Depo_dict['gauge1_address'],
    logger=logger
)
hipace_depo.connect()
print(f"✅ HiPace300 Depo connected on {HiPace300_Depo_dict['com_port']}")

2025-10-08 15:36:14 | INFO | Connecting to Pfeiffer device hipace_depo on COM37
2025-10-08 15:36:14 | INFO | Successfully connected to device at address 101


✅ HiPace300 Depo connected on COM37


In [22]:
# Initialize and connect HiPace450 (using HiPace300Bus class)
hipace450 = HiPace300Bus(
    device_id="hipace450",
    port=HiPace450_dict['com_port'],
    device_address=HiPace450_dict['device_address'],
    omnicontrol_address=HiPace450_dict['omnicontrol_address'],
    tc400_address=HiPace450_dict['tc400_address'],
    gauge1_address=HiPace450_dict['gauge1_address'],
    logger=logger
)
hipace450.connect()
print(f"✅ HiPace450 connected on {HiPace450_dict['com_port']} (initialized as HiPace300Bus)")

2025-10-08 15:42:08 | INFO | Connecting to Pfeiffer device hipace450 on COM34
2025-10-08 15:42:08 | INFO | Successfully connected to device at address 101


✅ HiPace450 connected on COM34 (initialized as HiPace300Bus)


In [9]:
# Initialize and connect TPG366 (5 pressure sensors)
tpg366 = TPG366(
    device_id="tpg366",
    port=TPG366_dict['com_port'],
    device_address=TPG366_dict['device_address'],
    logger=logger
)
tpg366.connect()
print(f"✅ TPG366 connected on {TPG366_dict['com_port']} with 5 sensors")

2025-10-08 15:36:17 | INFO | Connecting to Pfeiffer device tpg366 on COM22
2025-10-08 15:36:17 | INFO | Successfully connected to device at address 10


✅ TPG366 connected on COM22 with 5 sensors


In [ ]:
# Initialize and connect Chiller
chiller = Chiller(
    device_id="chiller",
    port=Chiller_dict['com_port'],
    device_address=Chiller_dict['device_address'],
    logger=logger
)
chiller.connect()
print(f"✅ Chiller connected on {Chiller_dict['com_port']}")

In [26]:
hipace450.get_operating_hours_pump()

5

In [14]:
chiller.read_temp()

17.08

## 5. Start Housekeeping for All Devices

In [27]:
# Start housekeeping for all pumps and chiller
hiscroll1.start_housekeeping()
hiscroll2.start_housekeeping()
hipace_transfer.start_housekeeping()
hipace_depo.start_housekeeping()
hipace450.start_housekeeping()
chiller.start_housekeeping()

print("✅ Housekeeping started for all devices:")
print("  - HiScroll1")
print("  - HiScroll2")
print("  - HiPace300 Transfer")
print("  - HiPace300 Depo")
print("  - HiPace450")
print("  - Chiller")
print("\nNote: TPG366 does not require housekeeping (continuous polling)")

2025-10-08 15:45:24 | INFO | Using external logger - no additional file logging needed
2025-10-08 15:45:24 | INFO | Housekeeping worker started for hiscroll1
2025-10-08 15:45:24 | INFO | Housekeeping started (internal mode) - interval: 30.0s
2025-10-08 15:45:24 | INFO | Using external logger - no additional file logging needed
2025-10-08 15:45:24 | INFO | Housekeeping worker started for hiscroll2
2025-10-08 15:45:24 | INFO | Housekeeping started (internal mode) - interval: 30.0s
2025-10-08 15:45:24 | INFO | Using external logger - no additional file logging needed
2025-10-08 15:45:24 | INFO | Housekeeping worker started for hipace_transfer
2025-10-08 15:45:24 | INFO | Housekeeping started (internal mode) - interval: 1.0s
2025-10-08 15:45:24 | INFO | Using external logger - no additional file logging needed
2025-10-08 15:45:24 | INFO | Housekeeping worker started for hipace_depo
2025-10-08 15:45:24 | INFO | Housekeeping started (internal mode) - interval: 1.0s
2025-10-08 15:45:24 | INFO

✅ Housekeeping started for all devices:
  - HiScroll1
  - HiScroll2
  - HiPace300 Transfer
  - HiPace300 Depo
  - HiPace450
  - Chiller

Note: TPG366 does not require housekeeping (continuous polling)


2025-10-08 15:45:24 | INFO | chiller   COM39   Cur_Temp   17.15//degC


## 5.1 Stop Housekeeping

In [36]:
hiscroll1.stop_housekeeping()
hiscroll2.stop_housekeeping()
hipace_transfer.stop_housekeeping()
hipace_depo.stop_housekeeping()
hipace450.stop_housekeeping()
chiller.stop_housekeeping()

2025-10-08 15:48:09 | INFO | Housekeeping stopped (internal mode)
2025-10-08 15:48:09 | INFO | Housekeeping stopped (internal mode)
2025-10-08 15:48:09 | INFO | Housekeeping stopped (internal mode)
2025-10-08 15:48:09 | INFO | Housekeeping stopped (internal mode)
2025-10-08 15:48:09 | INFO | Housekeeping stopped (internal mode)
2025-10-08 15:48:09 | INFO | Housekeeping stopped (internal mode)


True

## 6. Individual Pump Control

This section provides control cells for turning on/off each pump individually.

### 6.1 HiScroll Pump Control

In [ ]:
# Enable HiScroll1
hiscroll1.enable_pump()
print("✅ HiScroll1 enabled")

In [ ]:
# Disable HiScroll1
hiscroll1.disable_pump()
print("🛑 HiScroll1 disabled")

In [ ]:
# Enable HiScroll2
hiscroll2.enable_pump()
print("✅ HiScroll2 enabled")

In [ ]:
# Disable HiScroll2
hiscroll2.disable_pump()
print("🛑 HiScroll2 disabled")

### 6.2 Chiller Control


In [28]:
# Set chiller temperature (example: 16degC)
target_temp = 16.0
chiller.set_temperature(target_temp)
print(f"✅ Chiller temperature set to {target_temp}degC")

✅ Chiller temperature set to 16.0degC


In [29]:
# Start chiller
chiller.start_device()
print("✅ Chiller started")

✅ Chiller started


In [35]:
# Stop chiller
chiller.stop_device()
print("🛑 Chiller stopped")

🛑 Chiller stopped


In [30]:
# Check chiller status
current_temp = chiller.read_temp()
set_temp = chiller.read_set_temp()
status = chiller.read_status()
running = chiller.read_running()

print(f"Current Temperature: {current_temp}degC")
print(f"Set Temperature: {set_temp}degC")
print(f"Device Status: {status}")
print(f"Running State: {running}")

Current Temperature: 17.17degC
Set Temperature: 16.0degC
Device Status: OK
Running State: DEVICE RUNNING


### 6.3 HiPace300 Pump Control ALWAYS BOTH AT THE SAME TIME

In [ ]:
# Enable HiPace300 Transfer pump
hipace_transfer.enable_motor_pump()
# Enable HiPace300 Depo pump
hipace_depo.enable_motor_pump()
#print("✅ HiPace300 Transfer pump enabled")

In [ ]:
hipace_transfer.enable_pumpStatn()
hipace_depo.enable_pumpStatn()
print("✅ HiPace300 Depo pump enabled")

In [ ]:
# Disable HiPace300 Depo pump
hipace_depo.disable_pumpStatn()
print("🛑 HiPace300 Depo pump disabled")

# Disable HiPace300 Transfer pump
hipace_transfer.disable_pumpStatn()
print("🛑 HiPace300 Transfer pump disabled")

### 6.4 HiPace450 Pump Control

In [ ]:
# Enable HiPace450 pump
hipace450.enable_motor_pump()
hipace450.enable_pumpStatn()
print("✅ HiPace450 pump enabled")

In [ ]:
# Disable HiPace450 pump
hipace450.disable_pumpStatn()
print("🛑 HiPace450 pump disabled")

## 7. Live Plotting - Pressure Chain Visualization

This section provides live plotting for all pressure sensors and temperature monitoring.

In [31]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
from collections import deque

class PressureChainPlotter:
    """Live plotting class for complete ESIBD pressure chain monitoring"""
    
    def __init__(self, update_interval=1000):
        self.update_interval = update_interval
        
        # Data storage (keep last 100 points)
        self.timestamps = deque(maxlen=100)
        
        # Pressure data for 5 TPG366 sensors
        self.pressure_sensor1 = deque(maxlen=100)
        self.pressure_sensor2 = deque(maxlen=100)
        self.pressure_sensor3 = deque(maxlen=100)
        self.pressure_sensor4 = deque(maxlen=100)
        self.pressure_sensor5 = deque(maxlen=100)
        
        # Temperature data for HiScrolls
        self.hiscroll1_temp = deque(maxlen=100)
        self.hiscroll2_temp = deque(maxlen=100)
        
        # Temperature data for HiPaces
        self.hipace_transfer_temp = deque(maxlen=100)
        self.hipace_depo_temp = deque(maxlen=100)
        self.hipace450_temp = deque(maxlen=100)
        
        # Temperature data for Chiller
        self.chiller_current_temp = deque(maxlen=100)
        self.chiller_set_temp = deque(maxlen=100)
        
        # Create figure with 2 columns: left for 5 pressure plots, right for 3 temperature plots
        self.fig = plt.figure(figsize=(16, 12))
        
        # Create grid: 5 rows, 2 columns
        # Left column: 5 pressure sensors (shared x-axis)
        self.ax_p1 = plt.subplot(5, 2, 1)
        self.ax_p2 = plt.subplot(5, 2, 3, sharex=self.ax_p1)
        self.ax_p3 = plt.subplot(5, 2, 5, sharex=self.ax_p1)
        self.ax_p4 = plt.subplot(5, 2, 7, sharex=self.ax_p1)
        self.ax_p5 = plt.subplot(5, 2, 9, sharex=self.ax_p1)
        
        # Right column: 3 temperature plots (shared x-axis with pressure plots)
        self.ax_temp_hiscroll = plt.subplot(5, 2, 2, sharex=self.ax_p1)
        self.ax_temp_hipace = plt.subplot(5, 2, 4, sharex=self.ax_p1)
        self.ax_temp_chiller = plt.subplot(5, 2, 6, sharex=self.ax_p1)
    
    def update_plot(self, frame):
        try:
            current_time = datetime.now()
            self.timestamps.append(current_time)
            
            # Collect pressure data from TPG366 (5 sensors)
            try:
                self.pressure_sensor1.append(tpg366.read_pressure_value(1))
            except:
                self.pressure_sensor1.append(None)
            
            try:
                self.pressure_sensor2.append(tpg366.read_pressure_value(2))
            except:
                self.pressure_sensor2.append(None)
            
            try:
                self.pressure_sensor3.append(tpg366.read_pressure_value(3))
            except:
                self.pressure_sensor3.append(None)
            
            try:
                self.pressure_sensor4.append(tpg366.read_pressure_value(4))
            except:
                self.pressure_sensor4.append(None)
            
            try:
                self.pressure_sensor5.append(tpg366.read_pressure_value(5))
            except:
                self.pressure_sensor5.append(None)
            
            # Collect HiScroll temperature data
            try:
                self.hiscroll1_temp.append(hiscroll1.get_temp_motor())
            except:
                self.hiscroll1_temp.append(None)
            
            try:
                self.hiscroll2_temp.append(hiscroll2.get_temp_motor())
            except:
                self.hiscroll2_temp.append(None)
            
            # Collect HiPace temperature data
            try:
                self.hipace_transfer_temp.append(hipace_transfer.get_motor_temperature())
            except:
                self.hipace_transfer_temp.append(None)
            
            try:
                self.hipace_depo_temp.append(hipace_depo.get_motor_temperature())
            except:
                self.hipace_depo_temp.append(None)
            
            try:
                self.hipace450_temp.append(hipace450.get_motor_temperature())
            except:
                self.hipace450_temp.append(None)
            
            # Collect Chiller temperature data
            try:
                self.chiller_current_temp.append(chiller.read_temp())
            except:
                self.chiller_current_temp.append(None)
            
            try:
                self.chiller_set_temp.append(chiller.read_set_temp())
            except:
                self.chiller_set_temp.append(None)
            
        except Exception as e:
            logger.warning(f"Data collection error: {e}")
            return
        
        # Clear all axes
        self.ax_p1.clear()
        self.ax_p2.clear()
        self.ax_p3.clear()
        self.ax_p4.clear()
        self.ax_p5.clear()
        self.ax_temp_hiscroll.clear()
        self.ax_temp_hipace.clear()
        self.ax_temp_chiller.clear()
        
        # Convert deques to lists
        times = list(self.timestamps)
        
        # Only plot if we have data
        if not times:
            return
        
        try:
            # Plot Pressure Sensor 1
            p1_data = list(self.pressure_sensor1)
            valid_p1 = [(t, p) for t, p in zip(times, p1_data) if p is not None]
            if valid_p1:
                t1, p1 = zip(*valid_p1)
                self.ax_p1.plot(t1, p1, 'b-', marker='o', label='Sensor 1')
                self.ax_p1.set_yscale('log')
                self.ax_p1.set_ylabel('Pressure [mbar]')
                self.ax_p1.set_title('TPG366 Sensor 1 Pressure')
                self.ax_p1.legend(loc='upper right')
                self.ax_p1.grid(True)
            
            # Plot Pressure Sensor 2
            p2_data = list(self.pressure_sensor2)
            valid_p2 = [(t, p) for t, p in zip(times, p2_data) if p is not None]
            if valid_p2:
                t2, p2 = zip(*valid_p2)
                self.ax_p2.plot(t2, p2, 'r-', marker='s', label='Sensor 2')
                self.ax_p2.set_yscale('log')
                self.ax_p2.set_ylabel('Pressure [mbar]')
                self.ax_p2.set_title('TPG366 Sensor 2 Pressure')
                self.ax_p2.legend(loc='upper right')
                self.ax_p2.grid(True)
            
            # Plot Pressure Sensor 3
            p3_data = list(self.pressure_sensor3)
            valid_p3 = [(t, p) for t, p in zip(times, p3_data) if p is not None]
            if valid_p3:
                t3, p3 = zip(*valid_p3)
                self.ax_p3.plot(t3, p3, 'g-', marker='^', label='Sensor 3')
                self.ax_p3.set_yscale('log')
                self.ax_p3.set_ylabel('Pressure [mbar]')
                self.ax_p3.set_title('TPG366 Sensor 3 Pressure')
                self.ax_p3.legend(loc='upper right')
                self.ax_p3.grid(True)
            
            # Plot Pressure Sensor 4
            p4_data = list(self.pressure_sensor4)
            valid_p4 = [(t, p) for t, p in zip(times, p4_data) if p is not None]
            if valid_p4:
                t4, p4 = zip(*valid_p4)
                self.ax_p4.plot(t4, p4, 'm-', marker='d', label='Sensor 4')
                self.ax_p4.set_yscale('log')
                self.ax_p4.set_ylabel('Pressure [mbar]')
                self.ax_p4.set_title('TPG366 Sensor 4 Pressure')
                self.ax_p4.legend(loc='upper right')
                self.ax_p4.grid(True)
            
            # Plot Pressure Sensor 5
            p5_data = list(self.pressure_sensor5)
            valid_p5 = [(t, p) for t, p in zip(times, p5_data) if p is not None]
            if valid_p5:
                t5, p5 = zip(*valid_p5)
                self.ax_p5.plot(t5, p5, 'c-', marker='v', label='Sensor 5')
                self.ax_p5.set_yscale('log')
                self.ax_p5.set_ylabel('Pressure [mbar]')
                self.ax_p5.set_title('TPG366 Sensor 5 Pressure')
                self.ax_p5.legend(loc='upper right')
                self.ax_p5.grid(True)
            
            # Plot HiScroll Temperatures
            hs1_temps = list(self.hiscroll1_temp)
            hs2_temps = list(self.hiscroll2_temp)
            
            valid_hs1 = [(t, temp) for t, temp in zip(times, hs1_temps) if temp is not None]
            valid_hs2 = [(t, temp) for t, temp in zip(times, hs2_temps) if temp is not None]
            
            if valid_hs1:
                t_hs1, temp_hs1 = zip(*valid_hs1)
                self.ax_temp_hiscroll.plot(t_hs1, temp_hs1, 'b-', marker='o', label='HiScroll1')
            
            if valid_hs2:
                t_hs2, temp_hs2 = zip(*valid_hs2)
                self.ax_temp_hiscroll.plot(t_hs2, temp_hs2, 'r-', marker='s', label='HiScroll2')
            
            self.ax_temp_hiscroll.set_ylabel('Temperature [degC]')
            self.ax_temp_hiscroll.set_title('HiScroll Pump Temperatures')
            self.ax_temp_hiscroll.legend(loc='upper right')
            self.ax_temp_hiscroll.grid(True)
            
            # Plot HiPace Temperatures
            hp_t_temps = list(self.hipace_transfer_temp)
            hp_d_temps = list(self.hipace_depo_temp)
            hp_450_temps = list(self.hipace450_temp)
            
            valid_hp_t = [(t, temp) for t, temp in zip(times, hp_t_temps) if temp is not None]
            valid_hp_d = [(t, temp) for t, temp in zip(times, hp_d_temps) if temp is not None]
            valid_hp_450 = [(t, temp) for t, temp in zip(times, hp_450_temps) if temp is not None]
            
            if valid_hp_t:
                t_hp_t, temp_hp_t = zip(*valid_hp_t)
                self.ax_temp_hipace.plot(t_hp_t, temp_hp_t, 'g-', marker='^', label='HiPace Transfer')
            
            if valid_hp_d:
                t_hp_d, temp_hp_d = zip(*valid_hp_d)
                self.ax_temp_hipace.plot(t_hp_d, temp_hp_d, 'm-', marker='d', label='HiPace Depo')
            
            if valid_hp_450:
                t_hp_450, temp_hp_450 = zip(*valid_hp_450)
                self.ax_temp_hipace.plot(t_hp_450, temp_hp_450, 'orange', marker='s', label='HiPace450')
            
            self.ax_temp_hipace.set_ylabel('Temperature [degC]')
            self.ax_temp_hipace.set_xlabel('Time')
            self.ax_temp_hipace.set_title('HiPace Pump Temperatures')
            self.ax_temp_hipace.legend(loc='upper right')
            self.ax_temp_hipace.grid(True)
            
            # Plot Chiller Temperatures
            chiller_curr_temps = list(self.chiller_current_temp)
            chiller_set_temps = list(self.chiller_set_temp)
            
            valid_chiller_curr = [(t, temp) for t, temp in zip(times, chiller_curr_temps) if temp is not None]
            valid_chiller_set = [(t, temp) for t, temp in zip(times, chiller_set_temps) if temp is not None]
            
            if valid_chiller_curr:
                t_c_curr, temp_c_curr = zip(*valid_chiller_curr)
                self.ax_temp_chiller.plot(t_c_curr, temp_c_curr, 'c-', marker='o', label='Current Temp')
            
            if valid_chiller_set:
                t_c_set, temp_c_set = zip(*valid_chiller_set)
                self.ax_temp_chiller.plot(t_c_set, temp_c_set, 'c--', marker='s', alpha=0.7, label='Set Temp')
            
            self.ax_temp_chiller.set_ylabel('Temperature [degC]')
            self.ax_temp_chiller.set_xlabel('Time')
            self.ax_temp_chiller.set_title('Chiller Water Temperature')
            self.ax_temp_chiller.legend(loc='upper right')
            self.ax_temp_chiller.grid(True)
            
        except Exception as e:
            logger.error(f"Plotting error: {e}")
        
        # Format x-axis for all plots
        for ax in [self.ax_p1, self.ax_p2, self.ax_p3, self.ax_p4, self.ax_p5, 
                   self.ax_temp_hiscroll, self.ax_temp_hipace, self.ax_temp_chiller]:
            ax.tick_params(axis='x', rotation=45)
        
        # Hide x-tick labels for all except bottom plots (p5 on left, chiller on right)
        for ax in [self.ax_p1, self.ax_p2, self.ax_p3, self.ax_p4, 
                   self.ax_temp_hiscroll, self.ax_temp_hipace]:
            plt.setp(ax.get_xticklabels(), visible=False)
        
        self.fig.tight_layout()
    
    def start_live_plot(self):
        """Start live plotting"""
        self.ani = FuncAnimation(
            self.fig,
            self.update_plot,
            interval=self.update_interval,
            blit=False,
            cache_frame_data=False
        )
        
        plt.show()
        print("Live plotting started - Close window to stop")
    
    def stop_live_plot(self):
        """Stop live plotting"""
        if hasattr(self, 'ani'):
            self.ani.event_source.stop()
            print("Live plotting stopped.")
        else:
            print("No animation to stop.")

print("✅ Pressure chain plotter class ready")

✅ Pressure chain plotter class ready


In [32]:
%matplotlib notebook

In [33]:
# Create and start live plotting
plotter = PressureChainPlotter(update_interval=2000)  # Update every 2 seconds
plotter.start_live_plot()

<IPython.core.display.Javascript object>

Live plotting started - Close window to stop


In [34]:
# Stop live plotting
plotter.stop_live_plot()

Live plotting stopped.


## 7.5 Specialized Live Plot Windows

This section provides three separate plot windows for detailed monitoring of specific parameters.

In [ ]:
class PressureOnlyPlotter:
    """Live plotting class for all 5 pressure sensors"""
    
    def __init__(self, update_interval=2000):
        self.update_interval = update_interval
        
        # Data storage (keep last 100 points)
        self.timestamps = deque(maxlen=100)
        self.pressure_sensor1 = deque(maxlen=100)
        self.pressure_sensor2 = deque(maxlen=100)
        self.pressure_sensor3 = deque(maxlen=100)
        self.pressure_sensor4 = deque(maxlen=100)
        self.pressure_sensor5 = deque(maxlen=100)
        
        # Create figure with 5 subplots (sharing x-axis)
        self.fig, (self.ax1, self.ax2, self.ax3, self.ax4, self.ax5) = plt.subplots(
            5, 1, figsize=(10, 12), sharex=True
        )
        self.fig.suptitle('Pressure Monitoring - All 5 Sensors', fontsize=14)
    
    def update_plot(self, frame):
        try:
            current_time = datetime.now()
            self.timestamps.append(current_time)
            
            # Collect pressure data
            try:
                self.pressure_sensor1.append(tpg366.read_pressure_value(1))
            except:
                self.pressure_sensor1.append(None)
            
            try:
                self.pressure_sensor2.append(tpg366.read_pressure_value(2))
            except:
                self.pressure_sensor2.append(None)
            
            try:
                self.pressure_sensor3.append(tpg366.read_pressure_value(3))
            except:
                self.pressure_sensor3.append(None)
            
            try:
                self.pressure_sensor4.append(tpg366.read_pressure_value(4))
            except:
                self.pressure_sensor4.append(None)
            
            try:
                self.pressure_sensor5.append(tpg366.read_pressure_value(5))
            except:
                self.pressure_sensor5.append(None)
                
        except Exception as e:
            logger.warning(f"Data collection error: {e}")
            return
        
        # Clear all axes
        self.ax1.clear()
        self.ax2.clear()
        self.ax3.clear()
        self.ax4.clear()
        self.ax5.clear()
        
        times = list(self.timestamps)
        
        if not times:
            return
        
        try:
            # Plot each sensor
            p1_data = list(self.pressure_sensor1)
            valid_p1 = [(t, p) for t, p in zip(times, p1_data) if p is not None]
            if valid_p1:
                t1, p1 = zip(*valid_p1)
                self.ax1.plot(t1, p1, 'b-', marker='o')
                self.ax1.set_yscale('log')
                self.ax1.set_ylabel('Sensor 1 [mbar]')
                self.ax1.grid(True)
            
            p2_data = list(self.pressure_sensor2)
            valid_p2 = [(t, p) for t, p in zip(times, p2_data) if p is not None]
            if valid_p2:
                t2, p2 = zip(*valid_p2)
                self.ax2.plot(t2, p2, 'r-', marker='s')
                self.ax2.set_yscale('log')
                self.ax2.set_ylabel('Sensor 2 [mbar]')
                self.ax2.grid(True)
            
            p3_data = list(self.pressure_sensor3)
            valid_p3 = [(t, p) for t, p in zip(times, p3_data) if p is not None]
            if valid_p3:
                t3, p3 = zip(*valid_p3)
                self.ax3.plot(t3, p3, 'g-', marker='^')
                self.ax3.set_yscale('log')
                self.ax3.set_ylabel('Sensor 3 [mbar]')
                self.ax3.grid(True)
            
            p4_data = list(self.pressure_sensor4)
            valid_p4 = [(t, p) for t, p in zip(times, p4_data) if p is not None]
            if valid_p4:
                t4, p4 = zip(*valid_p4)
                self.ax4.plot(t4, p4, 'm-', marker='d')
                self.ax4.set_yscale('log')
                self.ax4.set_ylabel('Sensor 4 [mbar]')
                self.ax4.grid(True)
            
            p5_data = list(self.pressure_sensor5)
            valid_p5 = [(t, p) for t, p in zip(times, p5_data) if p is not None]
            if valid_p5:
                t5, p5 = zip(*valid_p5)
                self.ax5.plot(t5, p5, 'c-', marker='v')
                self.ax5.set_yscale('log')
                self.ax5.set_ylabel('Sensor 5 [mbar]')
                self.ax5.set_xlabel('Time')
                self.ax5.grid(True)
                
        except Exception as e:
            logger.error(f"Plotting error: {e}")
        
        # Format x-axis
        for ax in [self.ax1, self.ax2, self.ax3, self.ax4, self.ax5]:
            ax.tick_params(axis='x', rotation=45)
        
        # Hide x-tick labels for all except bottom
        for ax in [self.ax1, self.ax2, self.ax3, self.ax4]:
            plt.setp(ax.get_xticklabels(), visible=False)
        
        self.fig.tight_layout()
    
    def start_live_plot(self):
        """Start live plotting in separate window"""
        plt.ion()  # Turn on interactive mode
        
        self.ani = FuncAnimation(
            self.fig,
            self.update_plot,
            interval=self.update_interval,
            blit=False,
            cache_frame_data=False
        )
        
        plt.show(block=False)  # Non-blocking show
        print("Pressure-only live plotting started in separate window")
    
    def stop_live_plot(self):
        """Stop live plotting"""
        if hasattr(self, 'ani'):
            self.ani.event_source.stop()
            plt.close(self.fig)
            print("Pressure-only live plotting stopped.")
        else:
            print("No animation to stop.")

print("✅ Pressure-only plotter class ready")

In [ ]:
class HiPacePlotter:
    """Live plotting class for 3 HiPace pumps (Power, RPM, Temperature)"""
    
    def __init__(self, update_interval=2000):
        self.update_interval = update_interval
        
        # Data storage (keep last 100 points)
        self.timestamps = deque(maxlen=100)
        
        # HiPace Transfer data
        self.hipace_t_power = deque(maxlen=100)
        self.hipace_t_rpm = deque(maxlen=100)
        self.hipace_t_temp = deque(maxlen=100)
        
        # HiPace Depo data
        self.hipace_d_power = deque(maxlen=100)
        self.hipace_d_rpm = deque(maxlen=100)
        self.hipace_d_temp = deque(maxlen=100)
        
        # HiPace450 data
        self.hipace450_power = deque(maxlen=100)
        self.hipace450_rpm = deque(maxlen=100)
        self.hipace450_temp = deque(maxlen=100)
        
        # Create figure with 3 subplots (sharing x-axis)
        self.fig, (self.ax1, self.ax2, self.ax3) = plt.subplots(
            3, 1, figsize=(10, 10), sharex=True
        )
        self.fig.suptitle('HiPace Pumps Monitoring (Transfer, Depo, 450)', fontsize=14)
    
    def update_plot(self, frame):
        try:
            current_time = datetime.now()
            self.timestamps.append(current_time)
            
            # Collect HiPace Transfer data
            try:
                self.hipace_t_power.append(hipace_transfer.get_drive_power())
            except:
                self.hipace_t_power.append(None)
            
            try:
                self.hipace_t_rpm.append(hipace_transfer.get_actual_speed_rpm())
            except:
                self.hipace_t_rpm.append(None)
            
            try:
                self.hipace_t_temp.append(hipace_transfer.get_motor_temperature())
            except:
                self.hipace_t_temp.append(None)
            
            # Collect HiPace Depo data
            try:
                self.hipace_d_power.append(hipace_depo.get_drive_power())
            except:
                self.hipace_d_power.append(None)
            
            try:
                self.hipace_d_rpm.append(hipace_depo.get_actual_speed_rpm())
            except:
                self.hipace_d_rpm.append(None)
            
            try:
                self.hipace_d_temp.append(hipace_depo.get_motor_temperature())
            except:
                self.hipace_d_temp.append(None)
            
            # Collect HiPace450 data
            try:
                self.hipace450_power.append(hipace450.get_drive_power())
            except:
                self.hipace450_power.append(None)
            
            try:
                self.hipace450_rpm.append(hipace450.get_actual_speed_rpm())
            except:
                self.hipace450_rpm.append(None)
            
            try:
                self.hipace450_temp.append(hipace450.get_motor_temperature())
            except:
                self.hipace450_temp.append(None)
                
        except Exception as e:
            logger.warning(f"Data collection error: {e}")
            return
        
        # Clear all axes
        self.ax1.clear()
        self.ax2.clear()
        self.ax3.clear()
        
        times = list(self.timestamps)
        
        if not times:
            return
        
        try:
            # Plot 1: Power
            power_t = list(self.hipace_t_power)
            power_d = list(self.hipace_d_power)
            power_450 = list(self.hipace450_power)
            
            valid_t_power = [(t, p) for t, p in zip(times, power_t) if p is not None]
            valid_d_power = [(t, p) for t, p in zip(times, power_d) if p is not None]
            valid_450_power = [(t, p) for t, p in zip(times, power_450) if p is not None]
            
            if valid_t_power:
                t_t, p_t = zip(*valid_t_power)
                self.ax1.plot(t_t, p_t, 'g-', marker='^', label='Transfer')
            
            if valid_d_power:
                t_d, p_d = zip(*valid_d_power)
                self.ax1.plot(t_d, p_d, 'm-', marker='d', label='Depo')
            
            if valid_450_power:
                t_450, p_450 = zip(*valid_450_power)
                self.ax1.plot(t_450, p_450, 'orange', marker='s', label='HiPace450')
            
            self.ax1.set_ylabel('Power [W]')
            self.ax1.set_title('Drive Power')
            self.ax1.legend(loc='upper right')
            self.ax1.grid(True)
            
            # Plot 2: RPM
            rpm_t = list(self.hipace_t_rpm)
            rpm_d = list(self.hipace_d_rpm)
            rpm_450 = list(self.hipace450_rpm)
            
            valid_t_rpm = [(t, r) for t, r in zip(times, rpm_t) if r is not None]
            valid_d_rpm = [(t, r) for t, r in zip(times, rpm_d) if r is not None]
            valid_450_rpm = [(t, r) for t, r in zip(times, rpm_450) if r is not None]
            
            if valid_t_rpm:
                t_t, r_t = zip(*valid_t_rpm)
                self.ax2.plot(t_t, r_t, 'g-', marker='^', label='Transfer')
            
            if valid_d_rpm:
                t_d, r_d = zip(*valid_d_rpm)
                self.ax2.plot(t_d, r_d, 'm-', marker='d', label='Depo')
            
            if valid_450_rpm:
                t_450, r_450 = zip(*valid_450_rpm)
                self.ax2.plot(t_450, r_450, 'orange', marker='s', label='HiPace450')
            
            self.ax2.set_ylabel('RPM')
            self.ax2.set_title('Rotation Speed')
            self.ax2.legend(loc='upper right')
            self.ax2.grid(True)
            
            # Plot 3: Temperature
            temp_t = list(self.hipace_t_temp)
            temp_d = list(self.hipace_d_temp)
            temp_450 = list(self.hipace450_temp)
            
            valid_t_temp = [(t, temp) for t, temp in zip(times, temp_t) if temp is not None]
            valid_d_temp = [(t, temp) for t, temp in zip(times, temp_d) if temp is not None]
            valid_450_temp = [(t, temp) for t, temp in zip(times, temp_450) if temp is not None]
            
            if valid_t_temp:
                t_t, temp_t_val = zip(*valid_t_temp)
                self.ax3.plot(t_t, temp_t_val, 'g-', marker='^', label='Transfer')
            
            if valid_d_temp:
                t_d, temp_d_val = zip(*valid_d_temp)
                self.ax3.plot(t_d, temp_d_val, 'm-', marker='d', label='Depo')
            
            if valid_450_temp:
                t_450, temp_450_val = zip(*valid_450_temp)
                self.ax3.plot(t_450, temp_450_val, 'orange', marker='s', label='HiPace450')
            
            self.ax3.set_ylabel('Temperature [degC]')
            self.ax3.set_xlabel('Time')
            self.ax3.set_title('Motor Temperature')
            self.ax3.legend(loc='upper right')
            self.ax3.grid(True)
            
        except Exception as e:
            logger.error(f"Plotting error: {e}")
        
        # Format x-axis
        for ax in [self.ax1, self.ax2, self.ax3]:
            ax.tick_params(axis='x', rotation=45)
        
        # Hide x-tick labels for all except bottom
        for ax in [self.ax1, self.ax2]:
            plt.setp(ax.get_xticklabels(), visible=False)
        
        self.fig.tight_layout()
    
    def start_live_plot(self):
        """Start live plotting in separate window"""
        plt.ion()  # Turn on interactive mode
        
        self.ani = FuncAnimation(
            self.fig,
            self.update_plot,
            interval=self.update_interval,
            blit=False,
            cache_frame_data=False
        )
        
        plt.show(block=False)  # Non-blocking show
        print("HiPace pumps live plotting started in separate window")
    
    def stop_live_plot(self):
        """Stop live plotting"""
        if hasattr(self, 'ani'):
            self.ani.event_source.stop()
            plt.close(self.fig)
            print("HiPace pumps live plotting stopped.")
        else:
            print("No animation to stop.")

print("✅ HiPace plotter class ready")

In [ ]:
class HiScrollPlotter:
    """Live plotting class for 2 HiScroll pumps (Temperature and Power with dual y-axes)"""
    
    def __init__(self, update_interval=2000):
        self.update_interval = update_interval
        
        # Data storage (keep last 100 points)
        self.timestamps = deque(maxlen=100)
        
        # HiScroll1 data
        self.hiscroll1_temp = deque(maxlen=100)
        self.hiscroll1_power = deque(maxlen=100)
        
        # HiScroll2 data
        self.hiscroll2_temp = deque(maxlen=100)
        self.hiscroll2_power = deque(maxlen=100)
        
        # Create figure with 2 subplots (sharing x-axis)
        self.fig, (self.ax1, self.ax2) = plt.subplots(
            2, 1, figsize=(10, 8), sharex=True
        )
        self.fig.suptitle('HiScroll Pumps Monitoring (Temperature & Power)', fontsize=14)
    
    def update_plot(self, frame):
        try:
            current_time = datetime.now()
            self.timestamps.append(current_time)
            
            # Collect HiScroll1 data
            try:
                self.hiscroll1_temp.append(hiscroll1.get_temp_motor())
            except:
                self.hiscroll1_temp.append(None)
            
            try:
                self.hiscroll1_power.append(hiscroll1.get_drive_power())
            except:
                self.hiscroll1_power.append(None)
            
            # Collect HiScroll2 data
            try:
                self.hiscroll2_temp.append(hiscroll2.get_temp_motor())
            except:
                self.hiscroll2_temp.append(None)
            
            try:
                self.hiscroll2_power.append(hiscroll2.get_drive_power())
            except:
                self.hiscroll2_power.append(None)
                
        except Exception as e:
            logger.warning(f"Data collection error: {e}")
            return
        
        # Clear all axes
        self.ax1.clear()
        self.ax2.clear()
        
        times = list(self.timestamps)
        
        if not times:
            return
        
        try:
            # Plot 1: HiScroll1 with dual y-axes
            temp1 = list(self.hiscroll1_temp)
            power1 = list(self.hiscroll1_power)
            
            valid_temp1 = [(t, temp) for t, temp in zip(times, temp1) if temp is not None]
            valid_power1 = [(t, p) for t, p in zip(times, power1) if p is not None]
            
            # Left y-axis: Temperature
            if valid_temp1:
                t_temp1, temp1_val = zip(*valid_temp1)
                self.ax1.plot(t_temp1, temp1_val, 'b-', marker='o', label='Temperature')
            
            self.ax1.set_ylabel('Temperature [degC]', color='b')
            self.ax1.tick_params(axis='y', labelcolor='b')
            self.ax1.set_title('HiScroll1')
            self.ax1.grid(True, alpha=0.3)
            
            # Right y-axis: Power
            ax1_twin = self.ax1.twinx()
            if valid_power1:
                t_power1, power1_val = zip(*valid_power1)
                ax1_twin.plot(t_power1, power1_val, 'r--', marker='s', label='Power', alpha=0.7)
            
            ax1_twin.set_ylabel('Power [W]', color='r')
            ax1_twin.tick_params(axis='y', labelcolor='r')
            
            # Combine legends
            lines1, labels1 = self.ax1.get_legend_handles_labels()
            lines2, labels2 = ax1_twin.get_legend_handles_labels()
            self.ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
            
            # Plot 2: HiScroll2 with dual y-axes
            temp2 = list(self.hiscroll2_temp)
            power2 = list(self.hiscroll2_power)
            
            valid_temp2 = [(t, temp) for t, temp in zip(times, temp2) if temp is not None]
            valid_power2 = [(t, p) for t, p in zip(times, power2) if p is not None]
            
            # Left y-axis: Temperature
            if valid_temp2:
                t_temp2, temp2_val = zip(*valid_temp2)
                self.ax2.plot(t_temp2, temp2_val, 'b-', marker='o', label='Temperature')
            
            self.ax2.set_ylabel('Temperature [degC]', color='b')
            self.ax2.tick_params(axis='y', labelcolor='b')
            self.ax2.set_xlabel('Time')
            self.ax2.set_title('HiScroll2')
            self.ax2.grid(True, alpha=0.3)
            
            # Right y-axis: Power
            ax2_twin = self.ax2.twinx()
            if valid_power2:
                t_power2, power2_val = zip(*valid_power2)
                ax2_twin.plot(t_power2, power2_val, 'r--', marker='s', label='Power', alpha=0.7)
            
            ax2_twin.set_ylabel('Power [W]', color='r')
            ax2_twin.tick_params(axis='y', labelcolor='r')
            
            # Combine legends
            lines1, labels1 = self.ax2.get_legend_handles_labels()
            lines2, labels2 = ax2_twin.get_legend_handles_labels()
            self.ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
            
        except Exception as e:
            logger.error(f"Plotting error: {e}")
        
        # Format x-axis
        for ax in [self.ax1, self.ax2]:
            ax.tick_params(axis='x', rotation=45)
        
        # Hide x-tick labels for top plot
        plt.setp(self.ax1.get_xticklabels(), visible=False)
        
        self.fig.tight_layout()
    
    def start_live_plot(self):
        """Start live plotting in separate window"""
        plt.ion()  # Turn on interactive mode
        
        self.ani = FuncAnimation(
            self.fig,
            self.update_plot,
            interval=self.update_interval,
            blit=False,
            cache_frame_data=False
        )
        
        plt.show(block=False)  # Non-blocking show
        print("HiScroll pumps live plotting started in separate window")
    
    def stop_live_plot(self):
        """Stop live plotting"""
        if hasattr(self, 'ani'):
            self.ani.event_source.stop()
            plt.close(self.fig)
            print("HiScroll pumps live plotting stopped.")
        else:
            print("No animation to stop.")

print("✅ HiScroll plotter class ready")

In [ ]:
# Set matplotlib backend to Qt for separate windows
%matplotlib qt

### Start Individual Plot Windows

Run each cell below to open a separate plot window. All windows will update simultaneously and the kernel remains free for executing other cells.

In [ ]:
# Start Window 1: All Pressures (5 sensors)
pressure_plotter = PressureOnlyPlotter(update_interval=2000)
pressure_plotter.start_live_plot()

In [ ]:
# Start Window 2: HiPace Pumps (Power, RPM, Temperature)
hipace_plotter = HiPacePlotter(update_interval=2000)
hipace_plotter.start_live_plot()

In [ ]:
# Start Window 3: HiScroll Pumps (Temperature & Power with dual y-axes)
hiscroll_plotter = HiScrollPlotter(update_interval=2000)
hiscroll_plotter.start_live_plot()

### Stop Individual Plot Windows

Run the cells below to stop specific plot windows.

In [ ]:
# Stop pressure plotter
pressure_plotter.stop_live_plot()

In [ ]:
# Stop HiPace plotter
hipace_plotter.stop_live_plot()

In [ ]:
# Stop HiScroll plotter
hiscroll_plotter.stop_live_plot()

## 8. Data Logging and Export

Export collected data for further analysis.

In [ ]:
def export_pressure_chain_data(plotter, filename=None):
    """Export collected data to CSV file"""
    
    if filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"esibd_pressure_chain_{timestamp}.csv"
    
    # Create DataFrame from collected data
    data = {
        'timestamp': list(plotter.timestamps),
        'pressure_sensor1': list(plotter.pressure_sensor1),
        'pressure_sensor2': list(plotter.pressure_sensor2),
        'pressure_sensor3': list(plotter.pressure_sensor3),
        'pressure_sensor4': list(plotter.pressure_sensor4),
        'pressure_sensor5': list(plotter.pressure_sensor5),
        'hiscroll1_temp': list(plotter.hiscroll1_temp),
        'hiscroll2_temp': list(plotter.hiscroll2_temp),
        'hipace_transfer_temp': list(plotter.hipace_transfer_temp),
        'hipace_depo_temp': list(plotter.hipace_depo_temp),
        'hipace450_temp': list(plotter.hipace450_temp),
        'chiller_current_temp': list(plotter.chiller_current_temp),
        'chiller_set_temp': list(plotter.chiller_set_temp)
    }
    
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    logger.info(f"Data exported to {filename}")
    print(f"✅ Data exported to {filename}")
    print(f"   Total data points: {len(df)}")
    return df

print("✅ Data export function ready")

In [ ]:
# Export collected data
df = export_pressure_chain_data(plotter)

## 9. Stop Housekeeping and Disconnect Devices

In [ ]:
# Stop housekeeping for all devices
hiscroll1.stop_housekeeping()
hiscroll2.stop_housekeeping()
hipace_transfer.stop_housekeeping()
hipace_depo.stop_housekeeping()
chiller.stop_housekeeping()

print("✅ Housekeeping stopped for all devices")

In [37]:
# Disconnect all devices
print("Disconnecting all devices...")

try:
    hiscroll1.disconnect()
    print("  ✅ HiScroll1 disconnected")
except Exception as e:
    print(f"  ⚠️ HiScroll1 disconnect error: {e}")

try:
    hiscroll2.disconnect()
    print("  ✅ HiScroll2 disconnected")
except Exception as e:
    print(f"  ⚠️ HiScroll2 disconnect error: {e}")

try:
    hipace_transfer.disconnect()
    print("  ✅ HiPace300 Transfer disconnected")
except Exception as e:
    print(f"  ⚠️ HiPace300 Transfer disconnect error: {e}")

try:
    hipace_depo.disconnect()
    print("  ✅ HiPace300 Depo disconnected")
except Exception as e:
    print(f"  ⚠️ HiPace300 Depo disconnect error: {e}")

try:
    tpg366.disconnect()
    print("  ✅ TPG366 disconnected")
except Exception as e:
    print(f"  ⚠️ TPG366 disconnect error: {e}")

try:
    chiller.disconnect()
    print("  ✅ Chiller disconnected")
except Exception as e:
    print(f"  ⚠️ Chiller disconnect error: {e}")

print("\n✅ All devices disconnected")

2025-10-08 15:48:22 | INFO | Disconnected from Pfeiffer device hiscroll1


Disconnecting all devices...
  ✅ HiScroll1 disconnected


2025-10-08 15:48:22 | INFO | Disconnected from Pfeiffer device hiscroll2
2025-10-08 15:48:22 | INFO | Disconnected from Pfeiffer device hipace_transfer


  ✅ HiScroll2 disconnected
  ✅ HiPace300 Transfer disconnected


2025-10-08 15:48:23 | INFO | Disconnected from Pfeiffer device hipace_depo
2025-10-08 15:48:23 | INFO | Disconnected from Pfeiffer device tpg366
2025-10-08 15:48:23 | INFO | Disconnected from chiller chiller


  ✅ HiPace300 Depo disconnected
  ✅ TPG366 disconnected
  ✅ Chiller disconnected

✅ All devices disconnected


## Summary & Quick Reference

### Device Overview:
- **HiScroll1 & HiScroll2**: Backing pumps
- **HiPace300 Transfer**: Transfer chamber turbo pump
- **HiPace300 Depo**: Deposition chamber turbo pump
- **HiPace450**: Main high vacuum pump (placeholder - implementation needed)
- **TPG366**: Pressure controller with 5 sensors
- **Chiller**: Cooling unit

### Live Plotting Features:
- **5 Pressure Plots**: One for each TPG366 sensor (shared time axis, log scale)
- **HiScroll Temperature Plot**: Combined plot for both HiScroll pumps
- **HiPace Temperature Plot**: Combined plot for all HiPace pumps

### Key Operations:

#### Connect All Devices:
Run cells in Section 4 sequentially

#### Start Monitoring:
1. Start housekeeping (Section 5)
2. Enable live plotting (Section 7)

#### Control Individual Pumps:
- Section 6.1: HiScroll pump control
- Section 6.2: HiPace300 pump control

#### Export Data:
Run export cell in Section 8

#### Shutdown:
1. Stop live plotting
2. Stop housekeeping (Section 9)
3. Disconnect all devices (Section 9)

### Data Logging:
All device communications are logged to:
- File: `debugging/logs/esibd_pressure_chain_YYYYMMDD_HHMMSS.log`
- Format: Timestamp | Level | Module | Message

### Notes:
- Always start backing pumps (HiScrolls) before turbo pumps (HiPaces)
- Monitor pressure before enabling turbo pumps
- Wait for pressure to be low enough before starting turbo pumps
- HiPace450 device class needs to be implemented
- Update interval for live plotting can be adjusted (default: 2 seconds)